In [1]:
import time
import json
import sys
import pandas as pd
from pyspark.sql.functions import *
from datetime import datetime

In [2]:
import requests
requests.__version__

'2.25.1'

In [3]:
# Starting URL for Weather Map API Call.
url = "https://api.covid19api.com/summary"
print(url)

https://api.covid19api.com/summary


In [4]:
response = requests.get(url, stream=True)
response

<Response [200]>

In [5]:
response_json = response.json()



In [6]:
response_json.get("Global")

{'NewConfirmed': 1013577,
 'TotalConfirmed': 463412694,
 'NewDeaths': 5567,
 'TotalDeaths': 6054678,
 'NewRecovered': 0,
 'TotalRecovered': 0,
 'Date': '2022-03-17T15:04:36.417Z'}

In [7]:
covid_data = response_json
print(type(covid_data["Countries"]))
for value in covid_data["Countries"]:
    print (value["Country"], value["ID"])

<class 'list'>
Afghanistan 84938afb-68c5-4944-88a0-3d291a9273dd
Albania 356667ab-c3af-48d5-97d9-1158c34d7839
Algeria 266f37cd-5cf5-4975-b1ab-43b44c8f5d9d
Andorra d289fe2f-26e4-41eb-b478-51ab68531f53
Angola 28b762ec-883e-4994-a483-92921cb5e54b
Antarctica 88dd2f40-6590-434c-b983-6182f1f5fb5f
Antigua and Barbuda bdb7d7e2-a4c2-43fb-bdac-84a0ac03523a
Argentina e51aa6c5-b250-46d4-a200-b55bc180acc0
Armenia 8206f9d5-1e18-4f8d-a48a-d3e6b4b836d9
Australia dce1d496-f5f6-430b-8937-f2928429b2a1
Austria 83cb29c2-2780-4d5b-9aa9-be0663433379
Azerbaijan 2f9a4647-cffb-4efc-80f7-67ca4652e487
Bahamas c21812f2-051d-4ae0-b9dc-030cac4245d4
Bahrain 0f52a78a-f75f-4ff2-95cc-e9b06d23bd5a
Bangladesh aa8689b2-cc11-4145-8c80-5652bc002bc2
Barbados 991f1baf-2b8d-4cd3-82b0-e82e3263d100
Belarus 017f011a-07dc-4c44-817d-cb337a17b1a1
Belgium 79dd30f8-5ac5-4e4b-a546-2e2239782246
Belize d5fd7b2b-1c56-4d13-834f-3798013bb737
Benin ba777898-cb53-4a55-8da0-7050123f3a3a
Bhutan c572c574-3eae-4d91-87e4-c2b5ac9343b3
Bolivia 4b90461

In [8]:
print(len(covid_data["Countries"]))

194


In [9]:
for i in range (0, len(covid_data["Countries"])): 
        ID = covid_data["Countries"][i]["ID"]
        country = covid_data["Countries"][i]["Country"]
        country_code = covid_data["Countries"][i]["CountryCode"]
        Slug = covid_data["Countries"][i]["Slug"]
        New_Confirmed = covid_data["Countries"][i]["NewConfirmed"]
        Total_Confirmed= covid_data["Countries"][i]["TotalConfirmed"]
        New_Recovered = covid_data["Countries"][i]["NewRecovered"]
        Total_Recovered = covid_data["Countries"][i]["TotalRecovered"]
        Date = covid_data["Countries"][i]["Date"]
        Premium = covid_data["Countries"][i]["Premium"]
        print(f"{ID}: {country}: {country_code}: {Slug}: {New_Confirmed}: {Total_Confirmed}: {New_Recovered}: {Total_Recovered}: {Date}: {Premium}")
        print("----------------------------------------------------------------------------------------")

84938afb-68c5-4944-88a0-3d291a9273dd: Afghanistan: AF: afghanistan: 175: 176918: 0: 0: 2022-03-17T15:04:36.417Z: {}
----------------------------------------------------------------------------------------
356667ab-c3af-48d5-97d9-1158c34d7839: Albania: AL: albania: 0: 272804: 0: 0: 2022-03-17T15:04:36.417Z: {}
----------------------------------------------------------------------------------------
266f37cd-5cf5-4975-b1ab-43b44c8f5d9d: Algeria: DZ: algeria: 0: 265496: 0: 0: 2022-03-17T15:04:36.417Z: {}
----------------------------------------------------------------------------------------
d289fe2f-26e4-41eb-b478-51ab68531f53: Andorra: AD: andorra: 0: 38794: 0: 0: 2022-03-17T15:04:36.417Z: {}
----------------------------------------------------------------------------------------
28b762ec-883e-4994-a483-92921cb5e54b: Angola: AO: angola: 0: 98985: 0: 0: 2022-03-17T15:04:36.417Z: {}
----------------------------------------------------------------------------------------
88dd2f40-6590-434c-

In [10]:
# Create a dataframe.
df = pd.DataFrame(columns=["ID", "Country", "CountryCode", "Slug", "NewConfirmed", "TotalConfirmed", "NewRecovered", "TotalRecovered", "Date", "Premium"])


In [17]:
for i in range (0, len(covid_data["Countries"])): 
    currentItem = covid_data["Countries"][i]
    df.loc[i] = [covid_data["Countries"][i]["ID"],
                 covid_data["Countries"][i]["Country"],
                 covid_data["Countries"][i]["CountryCode"],
                 covid_data["Countries"][i]["Slug"],
                 covid_data["Countries"][i]["NewConfirmed"],
                 covid_data["Countries"][i]["TotalConfirmed"],
                 covid_data["Countries"][i]["NewRecovered"],
                 covid_data["Countries"][i]["TotalRecovered"],
                 covid_data["Countries"][i]["Date"],
                 covid_data["Countries"][i]["Premium"]]
df.head(10)

,ID,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewRecovered,TotalRecovered,Date,Premium
0,84938afb-68c5-4944-88a0-3d291a9273dd,Afghanistan,AF,afghanistan,175,176918,0,0,2022-03-17T15:04:36.417Z,{}
1,356667ab-c3af-48d5-97d9-1158c34d7839,Albania,AL,albania,0,272804,0,0,2022-03-17T15:04:36.417Z,{}
2,266f37cd-5cf5-4975-b1ab-43b44c8f5d9d,Algeria,DZ,algeria,0,265496,0,0,2022-03-17T15:04:36.417Z,{}
3,d289fe2f-26e4-41eb-b478-51ab68531f53,Andorra,AD,andorra,0,38794,0,0,2022-03-17T15:04:36.417Z,{}
4,28b762ec-883e-4994-a483-92921cb5e54b,Angola,AO,angola,0,98985,0,0,2022-03-17T15:04:36.417Z,{}
5,88dd2f40-6590-434c-b983-6182f1f5fb5f,Antarctica,AQ,antarctica,0,11,0,0,2022-03-17T15:04:36.417Z,{}
6,bdb7d7e2-a4c2-43fb-bdac-84a0ac03523a,Antigua and Barbuda,AG,antigua-and-barbuda,0,7470,0,0,2022-03-17T15:04:36.417Z,{}
7,e51aa6c5-b250-46d4-a200-b55bc180acc0,Argentina,AR,argentina,0,8985836,0,0,2022-03-17T15:04:36.417Z,{}
8,8206f9d5-1e18-4f8d-a48a-d3e6b4b836d9,Armenia,AM,armenia,47,422202,0,0,2022-03-17T15:04:36.417Z,{}
9,dce1d496-f5f6-430b-8937-f2928429b2a1,Australia,AU,australia,57148,3763166,0,0,2022-03-17T15:04:36.417Z,{}


In [19]:
df.dtypes

ID                object
Country           object
CountryCode       object
Slug              object
NewConfirmed      object
TotalConfirmed    object
NewRecovered      object
TotalRecovered    object
Date              object
Premium           object
dtype: object